In [1]:
import pandas as pd
import numpy as np

In [93]:
data = pd.read_csv('alz_data.csv')
data = data.drop(columns='Unnamed: 0')

In [92]:
data_z = data.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-19.1)/6.7))
data_z = data.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.3)/2.3))
data_z = data.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+30.9)/13.4))
data_z = data.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+82.2)/46.3))
data_z = data.assign(MoCA_Z = lambda x: ((x['moca']-26.3)/2.8))
data_z = data.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-21.4)/5.7))

In [94]:
data_sum = data.sum(axis = 1)
def alz_diagnose(data):
    diags = []
    for row in range(data_sum.shape[0]):
        row_sum = data.iloc[row].sum()
        try:
            if 1 <= row_sum <= 4:
                if data.loc[row,'GDS'] == 1:
                    diags.append('Normal')
                if data.loc[row, 'GDS'] == 2:
                     diags.append('SCD')
            if row_sum == 6.5:
                diags.append('MCI')
            if row_sum == 8:
                diags.append('Probable Dementia')
            if row_sum == 9:
                diags.append('Mild Dementia')
            if row_sum == 11:
                diags.append('Moderate Dementia')
            if row_sum == 13:
                diags.append('Severe Dementia')
            else:
                raise ValueError()
        except ValueError:
            diags.append('This combination of diagnostic values does not indicate clear diagnosis. Please verify values are correct.')
            pass
    return diags    

In [95]:
data_diag = alz_diagnose(data)
df5 = pd.DataFrame({'Diagnosis':data_diag})
data = data.join(df5)


In [96]:
print(data)

    GDS  CDR  Delayed Recall  Number Span Test Forward  Part A Trail Making  \
0     1  0.0              28                        10                   25   
1     1  0.0              30                        10                   24   
2     1  0.0              35                        10                   30   
3     1  0.0              28                        10                   30   
4     1  0.0              26                        12                   24   
..  ...  ...             ...                       ...                  ...   
95    6  3.0               1                         2                  150   
96    6  3.0               2                         2                  150   
97    7  3.0               1                         1                  150   
98    7  3.0               0                         0                  150   
99    7  3.0               0                         0                  150   

    Part B Trail Making  moca  Categories Animal  \